In [ ]:
import base
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rushd as rd
import scipy as sp
import seaborn as sns

# enables concurrent editing of base.py
from importlib import reload
reload(base)

sns.set_style('ticks')
sns.set_context('talk',rc={'font.family': 'sans-serif', 'font.sans-serif':['Helvetica Neue']})

## Setup

- Load data
- Add metadata
- Draw gates
- Gate transfected cells

Result from this section: DataFrame `df` representing transfected cells.

Load all lenti collection data collected as of 2024.03.27

In [ ]:
base_path = rd.datadir/'instruments'/'data'/'attune'/'kasey'/'2024.04.05_exp89'/'export'

plates = pd.DataFrame({
    'data_path': [base_path/'293T_control', base_path/'293T_plate1', base_path/'293T_plate2', base_path/'293T_plate3',
                  base_path/'MEF_3_plate1', base_path/'MEF_4-1_plate1', base_path/'MEF_4-1_plate2', base_path/'MEF_4-1_plate3'],
    'yaml_path': [base_path/'kasey_yaml'/'plate_control.yaml', base_path/'kasey_yaml'/'plate01.yaml', base_path/'kasey_yaml'/'plate02.yaml', base_path/'kasey_yaml'/'plate03.yaml',
                  base_path/'kasey_yaml'/'mef_3_plate01.yaml', base_path/'kasey_yaml'/'mef_4-1_plate01.yaml', base_path/'kasey_yaml'/'mef_4-1_plate02.yaml', base_path/'kasey_yaml'/'mef_4-1_plate03.yaml'],
})

output_path = rd.rootdir/'output'/'lenti_more'
cache_path = rd.rootdir/'output'/'lenti_BATS'/'data.gzip'

metadata_keys = set()
for p in plates['yaml_path'].unique():
    print(p)
    rd.plot.plot_well_metadata(p)
    metadata_keys.update(rd.flow.load_well_metadata(p).keys())
display(metadata_keys)

In [ ]:
# Load data
data = pd.DataFrame()
if cache_path.is_file(): data = pd.read_parquet(cache_path)
else: 
    channel_list = ['mCherry-A','mRuby2-A','FSC-A','SSC-A','mGL-A']
    data = rd.flow.load_groups_with_metadata(plates, columns=channel_list)

    # Remove negative channel values
    for c in channel_list: data = data[data[c]>0]
    
    data.dropna(inplace=True)
    data.to_parquet(rd.outfile(cache_path))
display(data)

In [ ]:
# Add metadata for constructs
metadata = base.get_metadata(rd.datadir/'projects'/'miR-iFFL'/'plasmids'/'construct-metadata.xlsx')
data = data.merge(metadata, how='left', on='construct')
display(data)

# Create dicts to specify colors/markers
metadata_dict = metadata.set_index('construct').to_dict('dict')
construct_palette = metadata_dict['color']
construct_markers = metadata_dict['markers']

In [ ]:
# Create color palette for comparing designs
metadata2 = metadata.copy()
metadata2.loc[(metadata2['group']=='marker'), 'color'] = 'black'
metadata2.loc[(metadata2['group']=='base'), 'color'] = 'black'

metadata2.loc[(metadata2['group']=='miR'), 'color'] = base.colors['purple']
metadata2.loc[(metadata2['group']=='miR') & (metadata2['miR_loc']=='UTR'), 'color'] = metadata2.loc[(metadata2['group']=='miR') & (metadata2['miR_loc']=='UTR'), 'color'].apply(base.get_light_color)

metadata2.loc[(metadata2['group'].isin(['ts3','ts5'])), 'color'] = base.colors['blue']
metadata2.loc[(metadata2['group'].isin(['ts3','ts5'])) & (metadata2['ts']=='FF6x1'), 'color'] = metadata2.loc[(metadata2['group'].isin(['ts3','ts5'])) & (metadata2['ts']=='FF6x1'), 'color'].apply(base.get_dark_color)
metadata2.loc[(metadata2['group'].isin(['ts3','ts5'])) & (metadata2['ts']=='FF4x1'), 'color'] = metadata2.loc[(metadata2['group'].isin(['ts3','ts5'])) & (metadata2['ts']=='FF4x1'), 'color'].apply(base.get_light_color)

metadata2.loc[(metadata2['group']=='controller') & (metadata2['design']==1), 'color'] = base.colors['teal']
metadata2.loc[(metadata2['group']=='controller') & (metadata2['design']==2), 'color'] = base.colors['orange']
metadata2.loc[(metadata2['group']=='controller') & (metadata2['design']==3), 'color'] = base.colors['red']

metadata2.loc[(metadata2['group']=='controller') & (metadata2['ts_kind']=='NT'), 'color'] = base.colors['gray']
metadata2.loc[(metadata2['group']=='controller') & (metadata2['ts_kind']=='NT') & (metadata2['design']==1), 'color'] = metadata2.loc[(metadata2['group']=='controller') & (metadata2['ts_kind']=='NT') & (metadata2['design']==1), 'color'].apply(base.get_light_color)
metadata2.loc[(metadata2['group']=='controller') & (metadata2['ts_kind']=='NT') & (metadata2['design']==3), 'color'] = metadata2.loc[(metadata2['group']=='controller') & (metadata2['ts_kind']=='NT') & (metadata2['design']==3), 'color'].apply(base.get_dark_color)

# markers
metadata2['markers'] = 'o'
metadata2.loc[(metadata2['group']=='base'), 'markers'] = 'X'
metadata2.loc[(metadata2['group']=='ts3'), 'markers'] = '^'
metadata2.loc[(metadata2['group']=='ts5'), 'markers'] = 'v'
metadata2.loc[(metadata2['miR'].isin(['miR.FF5','miR.FF4'])), 'markers'] = 'D'

# linestyles
metadata2['linestyle'] = '-' # default is solid
metadata2.loc[metadata2['group']=='marker', 'linestyle'] = ':'
metadata2.loc[(metadata2['group']=='ts5'), 'linestyle'] = '--'

metadata_dict2 = metadata2.set_index('construct').to_dict('dict')
designs_palette = metadata_dict2['color']
designs_markers = metadata_dict2['markers']

Draw gates based on ungated population, then gate infected cells.

Note that there were no uninfected MEF cells.

In [ ]:
gates = pd.DataFrame()
channel_list = ['mGL-A', 'mCherry-A', 'mRuby2-A']
for channel in channel_list:
    gates[channel] = data[data['construct']=='UT'].groupby(['cell'])[channel].apply(lambda x: x.quantile(0.999))
gates.reset_index(inplace=True)

# Add missing gates
gates.loc[len(gates.index)] = ['MEF',0,0,0,]  
gates.loc[gates['cell']=='MEF', channel_list] = gates.loc[gates['cell']=='293T', channel_list].values
gates['mGL-A'] = 2e3 # manually adjust

# Indicate which channels are relevant for each experiment
gates.sort_values(['cell'], inplace=True)
gates['marker'] = 'mGL-A'
gates['output'] = 'mRuby2-A'

display(gates)

Gate data per experiment based on marker expression

In [ ]:
def gate_data(df, gates):
    df = df.copy()
    cell = df['cell'].values[0] # the same for entire df, assuming df = data.groupby('cell')
    gates_dict = gates.set_index('cell').to_dict('dict') # format: column -> {index: value}
    marker = gates_dict['marker'][cell]
    df['expressing'] = df[marker] > gates_dict[marker][cell]
    df['marker'] = df[marker]
    df['output'] = df[gates_dict['output'][cell]]
    return df

data = data.groupby('cell')[data.columns].apply(lambda x: gate_data(x,gates))
data.reset_index(inplace=True, drop=True)
max = 1e6
df = data[(data['expressing']) & (data['output']<max)]
display(df)

In [ ]:
# Bin by marker quantiles
by = ['construct','cell','biorep','dox']
df['bin_marker_quantiles'] = df.groupby(by)['marker'].transform(lambda x: pd.qcut(x, q=20, duplicates='drop'))
quantiles = df.groupby(by+['bin_marker_quantiles'])['marker'].median().rename('bin_marker_quantiles_median').reset_index()
df_quantiles = df.merge(quantiles, how='left', on=by+['bin_marker_quantiles'])
display(df_quantiles)

In [ ]:
df_quantiles['marker'] = df_quantiles['marker'].astype(float)
df_quantiles['output'] = df_quantiles['output'].astype(float)

# Population stats
stat_list = [np.std, sp.stats.gmean, sp.stats.variation]
grouped = df_quantiles.groupby(by=['construct','biorep','cell','dox'])
stats = grouped[['marker','output']].agg(stat_list).reset_index().dropna()

# Rename columns as 'col_stat'
stats.columns = stats.columns.map(lambda i: base.rename_multilevel_cols(i))
stats['count'] = grouped['output'].count().reset_index()['output']
stats = stats.merge(metadata, how='left', on='construct')
display(stats)

In [ ]:
# Quantile stats & slope
df_quantiles['bin_marker_quantiles_median_log'] = df_quantiles['bin_marker_quantiles_median'].apply(np.log10)
df_quantiles['output_log'] = df_quantiles['output'].apply(np.log10)

stat_list = [sp.stats.gmean, ]#sp.stats.gstd, sp.stats.variation]
grouped = df_quantiles.groupby(by=['cell','construct','dox','biorep','bin_marker_quantiles_median'])
stats_quantiles = grouped[['marker','output']].agg(stat_list).reset_index().dropna()

# Rename columns as 'col_stat'
stats_quantiles.columns = stats_quantiles.columns.map(lambda i: base.rename_multilevel_cols(i))
stats_quantiles['count'] = grouped['output'].count().reset_index()['output']
stats_quantiles = stats_quantiles.merge(metadata, how='left', on='construct')
display(stats_quantiles)

stats_quantiles['bin_marker_quantiles_median_log'] = stats_quantiles['bin_marker_quantiles_median'].apply(np.log10)
stats_quantiles['output_gmean_log'] = stats_quantiles['output_gmean'].apply(np.log10)

In [ ]:
# Compute slope for all constructs
def get_slope(df):
    slope, intercept, r_value, p_value, stderr = sp.stats.linregress(df['bin_marker_quantiles_median_log'], df['output_gmean_log'])
    result = pd.DataFrame(columns=['slope', 'intercept_log', 'r_value', 'p_value', 'stderr'])
    result.loc[len(result.index)] = [slope, intercept, r_value, p_value, stderr]
    return result

fits = stats_quantiles.groupby(['cell','construct','dox','biorep'])[stats_quantiles.columns].apply(get_slope).reset_index()
fits['intercept'] = fits['intercept_log'].apply(lambda x: 10**x)
fits = fits.merge(metadata, how='left', on='construct')
display(fits)

### Distribution plots

In [ ]:
# MEF design 1
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (2e3,5e4)
ylim = (1e1,1e4)
biorep = 2
plot_df = df_quantiles[(df_quantiles['group'].isin(['base','controller'])) & (df_quantiles['design']<=1) & 
                       (df_quantiles['cell']=='293T') & (df_quantiles['dox']==1000) & (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=designs_palette, 
             legend=False, dashes=False, style='construct', markers=designs_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=designs_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='construct', palette=designs_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_293T-design1.svg')), bbox_inches='tight')

In [ ]:
# HEK293T design 2
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (2e3,5e4)
ylim = (1e1,1e4)
biorep = 2
plot_df = df_quantiles[(df_quantiles['group'].isin(['base','controller'])) & (df_quantiles['design'].isin([0,2])) & 
                       (df_quantiles['cell']=='293T') & (df_quantiles['dox']==1000) & (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=designs_palette, 
             legend=False, dashes=False, style='construct', markers=designs_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=designs_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='construct', palette=designs_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_293T-design2.svg')), bbox_inches='tight')

In [ ]:
# HEK293T design 3
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (2e3,5e4)
ylim = (1e1,1e4)
biorep = 2
plot_df = df_quantiles[(df_quantiles['group'].isin(['base','controller'])) & (df_quantiles['design'].isin([0,3])) & 
                       (df_quantiles['cell']=='293T') & (df_quantiles['dox']==1000) & (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=designs_palette, 
             legend=False, dashes=False, style='construct', markers=designs_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=designs_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='construct', palette=designs_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_293T-design3.svg')), bbox_inches='tight')

In [ ]:
# HEK293T designs 2&3
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (2e3,5e4)
ylim = (1e1,1e4)
biorep = 2
plot_df = df_quantiles[(df_quantiles['group'].isin(['base','controller'])) & (df_quantiles['design'].isin([0,2,3])) & 
                       (df_quantiles['cell']=='293T') & (df_quantiles['dox']==1000) & (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=designs_palette, 
             legend=False, dashes=False, style='construct', markers=designs_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=designs_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='construct', palette=designs_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_293T-designs2-3.svg')), bbox_inches='tight')

In [ ]:
# HEK293T miR only
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (2e3,5e4)
ylim = (1e1,1e4)
biorep = 2
plot_df = df_quantiles[(df_quantiles['group']=='miR') & 
                       (df_quantiles['cell']=='293T') & (df_quantiles['dox']==1000) & (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=designs_palette, 
             legend=False, dashes=False, style='construct', markers=designs_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=designs_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='construct', palette=designs_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_293T-miR-only.svg')), bbox_inches='tight')

In [ ]:
# HEK293T ts only
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (2e3,5e4)
ylim = (1e1,1e4)
biorep = 2
plot_df = df_quantiles[(df_quantiles['group'].isin(['ts3','ts5'])) & 
                       (df_quantiles['cell']=='293T') & (df_quantiles['dox']==1000) & (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=designs_palette, 
             legend=False, dashes=False, style='construct', markers=designs_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df[plot_df['group']=='ts5'], y='output', hue='construct', palette=designs_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls='--')
sns.kdeplot(data=plot_df[plot_df['group']=='ts3'], y='output', hue='construct', palette=designs_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls='-')
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='construct', palette=designs_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_293T-ts-only.svg')), bbox_inches='tight')

In [ ]:
# HEK293T designs -dox
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (2e3,5e4)
ylim = (1e1,1e4)
biorep = 2
plot_df = df_quantiles[(df_quantiles['group'].isin(['base','controller'])) & 
                       (df_quantiles['cell']=='293T') & (df_quantiles['dox']==0) & (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=designs_palette, 
             legend=False, dashes=False, style='construct', markers=designs_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=designs_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='construct', palette=designs_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_293T-designs-dox0.svg')), bbox_inches='tight')

In [ ]:
# MEF design 1
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (4e3,2e5)
ylim = (1e1,3e4)
biorep = 2
plot_df = df_quantiles[(df_quantiles['group'].isin(['base','controller'])) & (df_quantiles['design']<=1) & 
                       (df_quantiles['cell']=='MEF') & (df_quantiles['dox']==1000) & (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=designs_palette, 
             legend=False, dashes=False, style='construct', markers=designs_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=designs_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='construct', palette=designs_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_MEF-design1.svg')), bbox_inches='tight')

In [ ]:
# MEF design 2
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (4e3,2e5)
ylim = (1e1,3e4)
biorep = 2
plot_df = df_quantiles[(df_quantiles['group'].isin(['base','controller'])) & (df_quantiles['design'].isin([0,2])) & 
                       (df_quantiles['cell']=='MEF') & (df_quantiles['dox']==1000) & (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=designs_palette, 
             legend=False, dashes=False, style='construct', markers=designs_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=designs_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='construct', palette=designs_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_MEF-design2.svg')), bbox_inches='tight')

In [ ]:
# MEF design 3
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (4e3,2e5)
ylim = (1e1,3e4)
biorep = 2
plot_df = df_quantiles[(df_quantiles['group'].isin(['base','controller'])) & (df_quantiles['design'].isin([0,3])) & 
                       (df_quantiles['cell']=='MEF') & (df_quantiles['dox']==1000) & (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=designs_palette, 
             legend=False, dashes=False, style='construct', markers=designs_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=designs_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='construct', palette=designs_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_MEF-design3.svg')), bbox_inches='tight')

In [ ]:
# MEF designs 2&3
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (4e3,2e5)
ylim = (1e1,3e4)
biorep = 2
plot_df = df_quantiles[(df_quantiles['group'].isin(['base','controller'])) & (df_quantiles['design'].isin([0,2,3])) & 
                       (df_quantiles['cell']=='MEF') & (df_quantiles['dox']==1000) & (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=designs_palette, 
             legend=False, dashes=False, style='construct', markers=designs_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=designs_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='construct', palette=designs_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_MEF-designs2-3.svg')), bbox_inches='tight')

In [ ]:
# MEF miR only
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (4e3,2e5)
ylim = (1e1,3e4)
biorep = 2
plot_df = df_quantiles[(df_quantiles['group']=='miR') & 
                       (df_quantiles['cell']=='MEF') & (df_quantiles['dox']==1000) & (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=designs_palette, 
             legend=False, dashes=False, style='construct', markers=designs_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=designs_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='construct', palette=designs_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_MEF-miR-only.svg')), bbox_inches='tight')

In [ ]:
# MEF ts only
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (4e3,2e5)
ylim = (1e1,3e4)
biorep = 2
plot_df = df_quantiles[(df_quantiles['group'].isin(['ts3','ts5'])) & 
                       (df_quantiles['cell']=='MEF') & (df_quantiles['dox']==1000) & (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=designs_palette, 
             legend=False, dashes=False, style='construct', markers=designs_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df[plot_df['group']=='ts5'], y='output', hue='construct', palette=designs_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls='--')
sns.kdeplot(data=plot_df[plot_df['group']=='ts3'], y='output', hue='construct', palette=designs_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls='-')
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='construct', palette=designs_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_MEF-ts-only.svg')), bbox_inches='tight')

In [ ]:
# MEF designs -dox
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (2e3,2e5)
ylim = (1e1,3e4)
biorep = 2
plot_df = df_quantiles[(df_quantiles['group'].isin(['base','controller'])) & 
                       (df_quantiles['cell']=='MEF') & (df_quantiles['dox']==0) & (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=designs_palette, 
             legend=False, dashes=False, style='construct', markers=designs_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=designs_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='construct', palette=designs_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_MEF-designs-dox0.svg')), bbox_inches='tight')

### Stat plots

In [ ]:
# HEK293T all designs
fig, axes = plt.subplots(1,3, figsize=(10,5), gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.4))
ylim = (1e1,1e4)
plot_df = stats[((stats['group']=='controller') | (stats['group']=='base')) &
                (stats['cell']=='293T') & (stats['dox']==1000)].copy()
plot_df.sort_values('design', inplace=True)
plot_df2 = fits[((fits['group']=='controller') | (fits['group']=='base')) &
                (fits['cell']=='293T') & (fits['dox']==1000)].copy()
plot_df2.sort_values('design', inplace=True)

# stat gmean
ax = axes[0]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='design', y='output_gmean', hue='construct', palette=designs_palette,
                    legend=False, ax=ax, marker=designs_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Geometric mean', xlim=(-0.5, len(plot_df['design'].unique())-0.5), ylim=ylim, ylabel='', yscale='log',)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# stat std
ax = axes[1]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='design', y='output_std', hue='construct', palette=designs_palette,
                    legend=False, ax=ax, marker=designs_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Standard deviation', xlim=(-0.5, len(plot_df['design'].unique())-0.5), ylim=ylim, ylabel='', yscale='log',)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_std'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# slope
ax = axes[2]
for construct, group in plot_df2.groupby('construct'):
    sns.stripplot(data=group, x='design', y='slope', hue='construct', palette=designs_palette,
                    legend=False, ax=ax, marker=designs_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')

ax.set(title='Slope', xlim=(-0.5, len(plot_df2['design'].unique())-0.5), ylim=(0,1.2), ylabel='',)
sns.despine(ax=ax)
marker_baseline = fits.loc[(fits['group']=='marker'), 'slope'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

fig.savefig(rd.outfile(output_path/(f'stat_HEK293T-designs.svg')), bbox_inches='tight')

In [ ]:
# HEK293T all designs -dox
fig, axes = plt.subplots(1,3, figsize=(10,5), gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.4))
ylim = (1e1,1e4)
plot_df = stats[((stats['group']=='controller') | (stats['group']=='base')) &
                (stats['cell']=='293T') & (stats['dox']==0)].copy()
plot_df.sort_values('design', inplace=True)
plot_df2 = fits[((fits['group']=='controller') | (fits['group']=='base')) &
                (fits['cell']=='293T') & (fits['dox']==0)].copy()
plot_df2.sort_values('design', inplace=True)

# stat gmean
ax = axes[0]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='design', y='output_gmean', hue='construct', palette=designs_palette,
                    legend=False, ax=ax, marker=designs_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Geometric mean', xlim=(-0.5, len(plot_df['design'].unique())-0.5), ylim=ylim, ylabel='', yscale='log',)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# stat std
ax = axes[1]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='design', y='output_std', hue='construct', palette=designs_palette,
                    legend=False, ax=ax, marker=designs_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Standard deviation', xlim=(-0.5, len(plot_df['design'].unique())-0.5), ylim=ylim, ylabel='', yscale='log',)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_std'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# slope
ax = axes[2]
for construct, group in plot_df2.groupby('construct'):
    sns.stripplot(data=group, x='design', y='slope', hue='construct', palette=designs_palette,
                    legend=False, ax=ax, marker=designs_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')

ax.set(title='Slope', xlim=(-0.5, len(plot_df2['design'].unique())-0.5), ylim=(-0.2,1.2), ylabel='',)
sns.despine(ax=ax)
marker_baseline = fits.loc[(fits['group']=='marker'), 'slope'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

fig.savefig(rd.outfile(output_path/(f'stat_HEK293T-designs-dox0.svg')), bbox_inches='tight')

In [ ]:
# HEK293T controls
fig, axes = plt.subplots(1,3, figsize=(14,5), gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.2))
ylim = (1e1,1e4)
plot_df = stats[((stats['group'].isin(['base','miR','ts3','ts5']))) &
                (stats['cell']=='293T') & (stats['dox']==1000)].copy()
plot_df.sort_values(['group','ts_kind'], inplace=True)
plot_df2 = fits[((fits['group'].isin(['base','miR','ts3','ts5']))) &
                (fits['cell']=='293T') & (fits['dox']==1000)].copy()
plot_df2.sort_values(['group','ts_kind'], inplace=True)

# stat gmean
ax = axes[0]
for construct, group in plot_df.groupby('construct', sort=False):
    sns.stripplot(data=group, x='construct', y='output_gmean', hue='construct', palette=designs_palette,
                    legend=False, ax=ax, marker=designs_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Geometric mean', xlim=(-0.5, len(plot_df['construct'].unique())-0.5), ylim=ylim, ylabel='', yscale='log',)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')
rd.plot.generate_xticklabels(metadata2, 'construct', ['miR_loc','ts_loc'], annotate=True, ax=ax)

# stat std
ax = axes[1]
for construct, group in plot_df.groupby('construct', sort=False):
    sns.stripplot(data=group, x='construct', y='output_std', hue='construct', palette=designs_palette,
                    legend=False, ax=ax, marker=designs_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Standard deviation', xlim=(-0.5, len(plot_df['construct'].unique())-0.5), ylim=ylim, ylabel='', yscale='log',)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_std'].mean()
ax.axhline(marker_baseline, color='black', ls=':')
rd.plot.generate_xticklabels(metadata2, 'construct', ['miR_loc','ts_loc'], annotate=False, ax=ax)

# slope
ax = axes[2]
for construct, group in plot_df2.groupby('construct', sort=False):
    sns.stripplot(data=group, x='construct', y='slope', hue='construct', palette=designs_palette,
                    legend=False, ax=ax, marker=designs_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')

ax.set(title='Slope', xlim=(-0.5, len(plot_df2['construct'].unique())-0.5), ylim=(0,1.2), ylabel='',)
sns.despine(ax=ax)
marker_baseline = fits.loc[(fits['group']=='marker'), 'slope'].mean()
ax.axhline(marker_baseline, color='black', ls=':')
rd.plot.generate_xticklabels(metadata2, 'construct', ['miR_loc','ts_loc'], annotate=False, ax=ax)

fig.savefig(rd.outfile(output_path/(f'stat_HEK293T-controls.svg')), bbox_inches='tight')

In [ ]:
# HEK293T design 1
fig, axes = plt.subplots(1,3, figsize=(10,5), gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.4))
ylim = (1e1,1e4)
plot_df = stats[((stats['group'].isin(['controller','base'])) & (stats['design']<=1)) &
                (stats['cell']=='293T') & (stats['dox']==1000)].copy()
plot_df.sort_values('design', inplace=True)
plot_df2 = fits[((fits['group'].isin(['controller','base'])) & (fits['design']<=1)) &
                (fits['cell']=='293T') & (fits['dox']==1000)].copy()
plot_df2.sort_values('design', inplace=True)

# stat gmean
ax = axes[0]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='construct', y='output_gmean', hue='construct', palette=designs_palette,
                    legend=False, ax=ax, marker=designs_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Geometric mean', xlim=(-0.5, len(plot_df['construct'].unique())-0.5), ylim=ylim, ylabel='', yscale='log',)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')
rd.plot.generate_xticklabels(metadata2, 'construct', ['ts_kind'], annotate=False, ax=ax)

# stat std
ax = axes[1]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='construct', y='output_std', hue='construct', palette=designs_palette,
                    legend=False, ax=ax, marker=designs_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Standard deviation', xlim=(-0.5, len(plot_df['construct'].unique())-0.5), ylim=ylim, ylabel='', yscale='log',)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_std'].mean()
ax.axhline(marker_baseline, color='black', ls=':')
rd.plot.generate_xticklabels(metadata2, 'construct', ['ts_kind'], annotate=False, ax=ax)

# slope
ax = axes[2]
for construct, group in plot_df2.groupby('construct'):
    sns.stripplot(data=group, x='construct', y='slope', hue='construct', palette=designs_palette,
                    legend=False, ax=ax, marker=designs_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')

ax.set(title='Slope', xlim=(-0.5, len(plot_df2['construct'].unique())-0.5), ylim=(0,1.2), ylabel='',)
sns.despine(ax=ax)
marker_baseline = fits.loc[(fits['group']=='marker'), 'slope'].mean()
ax.axhline(marker_baseline, color='black', ls=':')
rd.plot.generate_xticklabels(metadata2, 'construct', ['ts_kind'], annotate=False, ax=ax)

fig.savefig(rd.outfile(output_path/(f'stat_HEK293T-design1.svg')), bbox_inches='tight')

In [ ]:
# MEF all designs
fig, axes = plt.subplots(1,3, figsize=(10,5), gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.4))
ylim = (1e1,3e4)
plot_df = stats[((stats['group']=='controller') | (stats['group']=='base')) &
                (stats['cell']=='MEF') & (stats['dox']==1000)].copy()
plot_df.sort_values('design', inplace=True)
plot_df2 = fits[((fits['group']=='controller') | (fits['group']=='base')) &
                (fits['cell']=='MEF') & (fits['dox']==1000)].copy()
plot_df2.sort_values('design', inplace=True)

# stat gmean
ax = axes[0]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='design', y='output_gmean', hue='construct', palette=designs_palette,
                    legend=False, ax=ax, marker=designs_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Geometric mean', xlim=(-0.5, len(plot_df['design'].unique())-0.5), ylim=ylim, ylabel='', yscale='log',)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# stat std
ax = axes[1]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='design', y='output_std', hue='construct', palette=designs_palette,
                    legend=False, ax=ax, marker=designs_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Standard deviation', xlim=(-0.5, len(plot_df['design'].unique())-0.5), ylim=ylim, ylabel='', yscale='log',)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_std'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# slope
ax = axes[2]
for construct, group in plot_df2.groupby('construct'):
    sns.stripplot(data=group, x='design', y='slope', hue='construct', palette=designs_palette,
                    legend=False, ax=ax, marker=designs_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')

ax.set(title='Slope', xlim=(-0.5, len(plot_df2['design'].unique())-0.5), ylim=(0,1.2), ylabel='',)
sns.despine(ax=ax)
marker_baseline = fits.loc[(fits['group']=='marker'), 'slope'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

fig.savefig(rd.outfile(output_path/(f'stat_MEF-designs.svg')), bbox_inches='tight')

In [ ]:
# MEF all designs -dox
fig, axes = plt.subplots(1,3, figsize=(10,5), gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.4))
ylim = (1e1,3e4)
plot_df = stats[((stats['group']=='controller') | (stats['group']=='base')) &
                (stats['cell']=='MEF') & (stats['dox']==0)].copy()
plot_df.sort_values('design', inplace=True)
plot_df2 = fits[((fits['group']=='controller') | (fits['group']=='base')) &
                (fits['cell']=='MEF') & (fits['dox']==0)].copy()
plot_df2.sort_values('design', inplace=True)

# stat gmean
ax = axes[0]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='design', y='output_gmean', hue='construct', palette=designs_palette,
                    legend=False, ax=ax, marker=designs_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Geometric mean', xlim=(-0.5, len(plot_df['design'].unique())-0.5), ylim=ylim, ylabel='', yscale='log',)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# stat std
ax = axes[1]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='design', y='output_std', hue='construct', palette=designs_palette,
                    legend=False, ax=ax, marker=designs_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Standard deviation', xlim=(-0.5, len(plot_df['design'].unique())-0.5), ylim=ylim, ylabel='', yscale='log',)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_std'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# slope
ax = axes[2]
for construct, group in plot_df2.groupby('construct'):
    sns.stripplot(data=group, x='design', y='slope', hue='construct', palette=designs_palette,
                    legend=False, ax=ax, marker=designs_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')

ax.set(title='Slope', xlim=(-0.5, len(plot_df2['design'].unique())-0.5), ylim=(-0.2,1.2), ylabel='',)
sns.despine(ax=ax)
marker_baseline = fits.loc[(fits['group']=='marker'), 'slope'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

fig.savefig(rd.outfile(output_path/(f'stat_MEF-designs-dox0.svg')), bbox_inches='tight')

In [ ]:
# MEF controls
fig, axes = plt.subplots(1,3, figsize=(14,5), gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.2))
ylim = (1e1,3e4)
plot_df = stats[((stats['group'].isin(['base','miR','ts3','ts5']))) &
                (stats['cell']=='MEF') & (stats['dox']==1000)].copy()
plot_df.sort_values(['group','ts_kind'], inplace=True)
plot_df2 = fits[((fits['group'].isin(['base','miR','ts3','ts5']))) &
                (fits['cell']=='MEF') & (fits['dox']==1000)].copy()
plot_df2.sort_values(['group','ts_kind'], inplace=True)

# stat gmean
ax = axes[0]
for construct, group in plot_df.groupby('construct', sort=False):
    sns.stripplot(data=group, x='construct', y='output_gmean', hue='construct', palette=designs_palette,
                    legend=False, ax=ax, marker=designs_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Geometric mean', xlim=(-0.5, len(plot_df['construct'].unique())-0.5), ylim=ylim, ylabel='', yscale='log',)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')
rd.plot.generate_xticklabels(metadata2, 'construct', ['miR_loc','ts_loc'], annotate=True, ax=ax)

# stat std
ax = axes[1]
for construct, group in plot_df.groupby('construct', sort=False):
    sns.stripplot(data=group, x='construct', y='output_std', hue='construct', palette=designs_palette,
                    legend=False, ax=ax, marker=designs_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Standard deviation', xlim=(-0.5, len(plot_df['construct'].unique())-0.5), ylim=ylim, ylabel='', yscale='log',)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_std'].mean()
ax.axhline(marker_baseline, color='black', ls=':')
rd.plot.generate_xticklabels(metadata2, 'construct', ['miR_loc','ts_loc'], annotate=False, ax=ax)

# slope
ax = axes[2]
for construct, group in plot_df2.groupby('construct', sort=False):
    sns.stripplot(data=group, x='construct', y='slope', hue='construct', palette=designs_palette,
                    legend=False, ax=ax, marker=designs_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')

ax.set(title='Slope', xlim=(-0.5, len(plot_df2['construct'].unique())-0.5), ylim=(0,1.2), ylabel='',)
sns.despine(ax=ax)
marker_baseline = fits.loc[(fits['group']=='marker'), 'slope'].mean()
ax.axhline(marker_baseline, color='black', ls=':')
rd.plot.generate_xticklabels(metadata2, 'construct', ['miR_loc','ts_loc'], annotate=False, ax=ax)

fig.savefig(rd.outfile(output_path/(f'stat_MEF-controls.svg')), bbox_inches='tight')

In [ ]:
# MEF design 1
fig, axes = plt.subplots(1,3, figsize=(10,5), gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.4))
ylim = (1e1,3e4)
plot_df = stats[((stats['group'].isin(['controller','base'])) & (stats['design']<=1)) &
                (stats['cell']=='MEF') & (stats['dox']==1000)].copy()
plot_df.sort_values('design', inplace=True)
plot_df2 = fits[((fits['group'].isin(['controller','base'])) & (fits['design']<=1)) &
                (fits['cell']=='MEF') & (fits['dox']==1000)].copy()
plot_df2.sort_values('design', inplace=True)

# stat gmean
ax = axes[0]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='construct', y='output_gmean', hue='construct', palette=designs_palette,
                    legend=False, ax=ax, marker=designs_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Geometric mean', xlim=(-0.5, len(plot_df['construct'].unique())-0.5), ylim=ylim, ylabel='', yscale='log',)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')
rd.plot.generate_xticklabels(metadata2, 'construct', ['ts_kind'], annotate=False, ax=ax)

# stat std
ax = axes[1]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='construct', y='output_std', hue='construct', palette=designs_palette,
                    legend=False, ax=ax, marker=designs_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Standard deviation', xlim=(-0.5, len(plot_df['construct'].unique())-0.5), ylim=ylim, ylabel='', yscale='log',)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_std'].mean()
ax.axhline(marker_baseline, color='black', ls=':')
rd.plot.generate_xticklabels(metadata2, 'construct', ['ts_kind'], annotate=False, ax=ax)

# slope
ax = axes[2]
for construct, group in plot_df2.groupby('construct'):
    sns.stripplot(data=group, x='construct', y='slope', hue='construct', palette=designs_palette,
                    legend=False, ax=ax, marker=designs_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')

ax.set(title='Slope', xlim=(-0.5, len(plot_df2['construct'].unique())-0.5), ylim=(0,1.2), ylabel='',)
sns.despine(ax=ax)
marker_baseline = fits.loc[(fits['group']=='marker'), 'slope'].mean()
ax.axhline(marker_baseline, color='black', ls=':')
rd.plot.generate_xticklabels(metadata2, 'construct', ['ts_kind'], annotate=False, ax=ax)

fig.savefig(rd.outfile(output_path/(f'stat_MEF-design1.svg')), bbox_inches='tight')